In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")


In [3]:
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [4]:
import ast
import json

# Function to safely parse the string as a list
def parse_list_string(s):
    try:
        # First try to parse as JSON
        return json.loads(s)
    except:
        try:
            # Then try to parse as Python literal
            return ast.literal_eval(s)
        except:
            # If all fails, return as is
            return [s]

# Create a new dataframe to store the expanded data
expanded_rows = []

for _, row in df.iterrows():
    # Parse the prompts and responses
    prompts = parse_list_string(row['prompt'])
    responses_a = parse_list_string(row['response_a'])
    responses_b = parse_list_string(row['response_b'])
    
    # Make sure all lists have the same length
    max_len = max(len(prompts), len(responses_a), len(responses_b))
    prompts = prompts + [''] * (max_len - len(prompts))
    responses_a = responses_a + [''] * (max_len - len(responses_a))
    responses_b = responses_b + [''] * (max_len - len(responses_b))
    
    # Create a new row for each prompt-response pair
    for i in range(max_len):
        expanded_rows.append({
            'id': row['id'],
            'model_a': row['model_a'],
            'model_b': row['model_b'],
            'prompt': prompts[i],
            'response_a': responses_a[i],
            'response_b': responses_b[i],
            'winner_model_a': row['winner_model_a'],
            'winner_model_b': row['winner_model_b'],
            'winner_tie': row['winner_tie']
        })

# Create the expanded dataframe
expanded_df = pd.DataFrame(expanded_rows)

# Display the first few rows of the expanded dataframe
expanded_df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0
1,30192,gpt-4-1106-preview,gpt-4-0613,"OK, does pineapple belong on a pizza? Relax an...","Ah, the age-old culinary conundrum that has di...","As an AI, I don't eat, so I don't have persona...",1,0,0
2,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0
3,53567,koala-13b,gpt-4-0613,How can I get both of them as quick as possibl...,If you want to get both a marriage license and...,"In California, here are the general steps to o...",0,1,0
4,53567,koala-13b,gpt-4-0613,What is the minimal time to get them? 1 day or...,The minimal time to get a marriage license and...,The marriage license in California is issued i...,0,1,0


In [12]:
filtered_df = expanded_df[expanded_df['winner_tie'] == 0]
filtered_df = filtered_df.drop(columns=['winner_tie', 'id'])
print(f"Original shape: {expanded_df.shape}")
print(f"After removing ties: {filtered_df.shape}")
print(filtered_df[['winner_model_a', 'winner_model_b']].value_counts())

Original shape: (71514, 9)
After removing ties: (49865, 7)
winner_model_a  winner_model_b
1               0                 25181
0               1                 24684
Name: count, dtype: int64


In [13]:
filtered_df.head()

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b
0,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0
1,gpt-4-1106-preview,gpt-4-0613,"OK, does pineapple belong on a pizza? Relax an...","Ah, the age-old culinary conundrum that has di...","As an AI, I don't eat, so I don't have persona...",1,0
2,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1
3,koala-13b,gpt-4-0613,How can I get both of them as quick as possibl...,If you want to get both a marriage license and...,"In California, here are the general steps to o...",0,1
4,koala-13b,gpt-4-0613,What is the minimal time to get them? 1 day or...,The minimal time to get a marriage license and...,The marriage license in California is issued i...,0,1


In [14]:
filtered_df.to_csv("filtered_results.csv", index=False)

In [15]:
test = pd.read_csv("filtered_results.csv")
test.head()


,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b
0,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0
1,gpt-4-1106-preview,gpt-4-0613,"OK, does pineapple belong on a pizza? Relax an...","Ah, the age-old culinary conundrum that has di...","As an AI, I don't eat, so I don't have persona...",1,0
2,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1
3,koala-13b,gpt-4-0613,How can I get both of them as quick as possibl...,If you want to get both a marriage license and...,"In California, here are the general steps to o...",0,1
4,koala-13b,gpt-4-0613,What is the minimal time to get them? 1 day or...,The minimal time to get a marriage license and...,The marriage license in California is issued i...,0,1
